# Sweep Params for MatrixFactorization

This notebook is used to sweep the K parameters in our Matrix Factorization model.


Input: a 20000 by 426 rating matrix.

Output: a json file that contains keys (K, reg) mapped to training MSE and test MSE's.

The json file "matrix_factorization_all_params.json" contains all the training and test MSE's that we swept.
The params we swept range from:
1. k = 50 - 130
2. reg = 1 - 10
3. niter = 20

In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.linalg as la
import gc


In [2]:
path = "../ShrinkMatrices/"
npz_filename = path + "rating_matrix_shrunk.npz"

In [3]:
sparse_rating_matrix = sp.load_npz(npz_filename)
sparse_rating_matrix = sparse_rating_matrix.astype("int8")
dense_rating_matrix = sparse_rating_matrix.todense()

In [136]:
dense_rating_matrix.shape

(1000, 14473)

In [137]:
np.count_nonzero(dense_rating_matrix)

44554

In [138]:
np.count_nonzero(dense_rating_matrix) / (dense_rating_matrix.shape[0] * dense_rating_matrix.shape[1])

0.0030784218890347543

## Train Test Split

Method: randomly select 10% of user-book pairs that have nonzero entry as test setthe set is train set

In [4]:
# X_tr now is 1d array version of dense_rating_matrix
X_tr = np.asarray(dense_rating_matrix.copy())
X_tr = X_tr.flatten()

nonzero_pairs = np.nonzero(X_tr)[0]
num_non_zero_pairs = len(nonzero_pairs)

total_num_pairs = X_tr.shape[0]
num_testing_pairs = int(0.1 * num_non_zero_pairs)

# seeds the random generator
np.random.seed(0)

# indices of 1d array X_tr
testing_pair_indices = np.random.choice(nonzero_pairs, num_testing_pairs, replace=False)
training_pair_indices = list(set(np.arange(total_num_pairs)) - set(testing_pair_indices))

In [5]:
len(training_pair_indices)

8503229

In [6]:
X_te = X_tr.copy()

# sets testing pairs in training set to be 0
X_tr[testing_pair_indices] = 0

# sets training pairs in testing set to be 0
X_te[training_pair_indices] = 0

In [7]:
np.count_nonzero(X_tr)

150939

In [8]:
np.count_nonzero(X_te)

16771

In [9]:
# takes X_tr and X_te back to shape of dense_rating_matrix

X_tr = X_tr.reshape((dense_rating_matrix.shape[0], dense_rating_matrix.shape[1]))
X_te = X_te.reshape((dense_rating_matrix.shape[0], dense_rating_matrix.shape[1]))

In [10]:
np.count_nonzero(X_tr)

150939

In [11]:
np.count_nonzero(X_te)

16771

In [12]:
P = None
dense_rating_matrix = None
sparse_rating_matrix = None
gc.collect()

7

## Error and Train Functions

1. Our error function is a simple MSE function.
2. Our train function uses alternating least squares to find the most optimal UV to decompose our rating matrix.

In [20]:
def error(X, nnz_indices, U, V, reg, verbose=False):
    """ Compute the mean error of the observed ratings in X and their estimated values. 
        Args: 
            X (numpy 2D array) : a ratings matrix as specified above 
            nnz_indices (numpy 2D array) : nonzero indices of dense_rating_matrix
            U (numpy 2D array) : a matrix of features for each user (m x k)
            V (numpy 2D array) : a matrix of features for each movie (k x n)
            reg (float): regularization parameter
        Returns: 
            (float) : the mean squared error of the observed ratings with their estimated values with penalty for u and v coefficients
        """

    num_nnz = np.sum(nnz_indices)
    term_1 = np.sum(np.multiply(nnz_indices, ((np.matmul(U, V) - X) ** 2)))
    term_2 = reg * np.sum(U * U)
    term_3 = reg * np.sum(V * V)

    return (term_1 + term_2 + term_3) / num_nnz

train_list = []
test_list = []
def train(X_tr, X_te, U_init, V_init, k, niters=10, reg=1, verbose=False):
    """ Train a collaborative filtering model. 
        Args: 
            X_tr (numpy 2D array) : the training ratings matrix as specified above
            X_te (numpy 2D array) : the testing ratings matrix as specified above
            U_init (numpy 2D array) : an initial matrix of features for each user
            V_init (numpy 2D array) : an initial matrix of features for each movie
            k (int) : the number of features used in the CF model
            niters (int) : number of iterations to run
            reg (float) : regularization parameter
            verbose (boolean) : verbosity flag for printing useful messages
            
        Returns:
            (U,V) : A pair of the resulting learned matrix factorization
    """
    
    U = np.empty((X_tr.shape[0], k))
    V = np.empty((k, X_tr.shape[1]))

    num_users, num_items = X_tr.shape
    reg_matrix = reg * np.eye(k,k)
    
    nnz_indices = (X_tr != 0)
    nnz_indices_te = (X_te != 0)
    
    for iteration in range(1, niters + 1):
        
        # fix V and update U
        for i in range(num_users):
            idx = nnz_indices[i,:]
            lhs = V[:,idx]
            rhs = np.matmul(lhs, X_tr[i, idx].T)
            lhs = np.matmul(lhs, lhs.T) + reg_matrix
            U[i,:] = np.linalg.solve(lhs, rhs).T

        # fix U and update V
        for j in range(num_items):
            idx = nnz_indices[:,j]
            lhs = U[idx,:]
            rhs = np.matmul(lhs.T, X_tr[idx, j])
            lhs = np.matmul(lhs.T, lhs) + reg_matrix
            V[:,j] = np.linalg.solve(lhs, rhs)
        
        # save some memory
        idx = None
        lhs = None
        rhs = None
        gc.collect()
        
        print ("--------------------------------------------------")
        print ("Iteration: " + str(iteration))
        
        
        training_error = error(X_tr, nnz_indices, U, V, 0, verbose)
        testing_error = error(X_te, nnz_indices_te, U, V, 0, verbose)
        if verbose:
            train_list.append(training_error)
            test_list.append(testing_error)
            print ("Training error: " + str(training_error))
            print ("Testing error: " + str(testing_error))
        
        print ("--------------------------------------------------")
            
    return (training_error, testing_error)

## Run the Model using the K specified 

Here is where you can change your params to speicify the params that you want to sweep.

In [213]:
k_range = np.concatenate([np.arange(20, 41), np.arange(81, 101)])

Because we have prepared 30 notebooks to sweep parameters in parrallel, here is a simple calculation that we employed to see which notebook to sweep which parameters.

In [214]:

if_write_file = True

notebook_num = 3
total_notebook_num = 15

k_per_notebook = int(len(k_range) / 15) + 1

k_range = k_range[(k_per_notebook * (notebook_num - 1)):(k_per_notebook * notebook_num)]

k_range

array([20, 21, 22])

Here the model starts training and sweeping through the params.

In [218]:
parameter_dict = {}
niters = 20
reg_range = np.arange(1, 26)

for k in k_range:

    # initialize U and V
    # these will not be modified
    U_init = np.apply_along_axis(lambda x: x/sum(x), 1, abs(np.random.randn(X_tr.shape[0],k)))
    V_init = np.apply_along_axis(lambda x: x/sum(x), 1, abs(np.random.randn(k,X_tr.shape[1])))
            
    for reg in reg_range:
        parameter_dict[str((k, reg))] = train(X_tr, X_te, U_init, V_init, k, niters, reg, verbose=False)
        print (str((k, reg)) + " " + str(parameter_dict[str((k, reg))]))
    
    # write to file for every k
    
    if if_write_file:
        with open(str(notebook_num) + ".json", "w+") as f:
            json.dump(parameter_dict, f)
            print ("k = " + str(k) + " saved")
            print ("")
    
print ("Complete")


(20, 1) (0.10157804962606012, 3.2723173993536854)
k = 20 saved

(21, 1) (0.2756627581117208, 2.3634534367614677)
k = 21 saved

(22, 1) (0.2652228751573767, 2.3544869122944485)
k = 22 saved

Complete


## Run Model with Best Params
Our best params are
1. k = 80
2. reg = 5.0

In [21]:
k = 80
reg = 5.0
U_init = np.apply_along_axis(lambda x: x/sum(x), 1, abs(np.random.randn(X_tr.shape[0],k)))
V_init = np.apply_along_axis(lambda x: x/sum(x), 1, abs(np.random.randn(k,X_tr.shape[1])))
train(X_tr, X_te, U_init, V_init, k, niters = 50, reg = reg, verbose=True)

--------------------------------------------------
Iteration: 1
Training error: 0.14786547414005335
Testing error: 1.508293806924531
--------------------------------------------------
--------------------------------------------------
Iteration: 2
Training error: 0.08154031858926152
Testing error: 1.3588202682288506
--------------------------------------------------
--------------------------------------------------
Iteration: 3
Training error: 0.06524791997434765
Testing error: 1.2924817885412647
--------------------------------------------------
--------------------------------------------------
Iteration: 4
Training error: 0.058639412653340994
Testing error: 1.2476283969129145
--------------------------------------------------
--------------------------------------------------
Iteration: 5
Training error: 0.055477872743219
Testing error: 1.2182995997716797
--------------------------------------------------
--------------------------------------------------
Iteration: 6
Training erro

Training error: 0.05002358818094862
Testing error: 1.1566253667500017
--------------------------------------------------
--------------------------------------------------
Iteration: 46
Training error: 0.050024131669658446
Testing error: 1.1565922732069258
--------------------------------------------------
--------------------------------------------------
Iteration: 47
Training error: 0.050024702841462085
Testing error: 1.156560714258855
--------------------------------------------------
--------------------------------------------------
Iteration: 48
Training error: 0.05002529232893524
Testing error: 1.1565306213873117
--------------------------------------------------
--------------------------------------------------
Iteration: 49
Training error: 0.05002589228171316
Testing error: 1.1565019286841964
--------------------------------------------------
--------------------------------------------------
Iteration: 50
Training error: 0.05002649614114599
Testing error: 1.1564745726702201

(0.05002649614114599, 1.1564745726702201)